In [255]:
import numpy as np
import pandas as pd
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score, accuracy_score

import joblib
from joblib import dump, load

pd.options.display.max_columns = 400
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 600
pd.options.display.precision = 10

In [256]:
df_train = pd.read_excel("C:/Users/tjdal/Desktop/fast_campus/Project/individual_project/Crowdflower_Search/__data/excel/train.xlsx").fillna("")

In [257]:
df_test = pd.read_excel("C:/Users/tjdal/Desktop/fast_campus/Project/individual_project/Crowdflower_Search/__data/excel/test.xlsx").fillna("")

In [258]:
test_sub = pd.read_excel("C:/Users/tjdal/Desktop/fast_campus/Project/individual_project/Crowdflower_Search/__data/excel/sampleSubmission.xlsx")

In [259]:
df_train = df_train.drop('Unnamed: 6', axis = 1)
df_train = df_train.drop('Unnamed: 7', axis = 1)
df_train = df_train.drop('relevance_variance', axis = 1)

In [260]:
df_test = df_test.drop('Unnamed: 4', axis = 1)
df_test = df_test.drop('Unnamed: 5', axis = 1)

### df_test / test_sub matching

In [261]:
print(len(df_test))
print(len(test_sub))

22511
22513


In [262]:
# submission 에는 있지만 test data 에는 존재하지 않는 'id' 확인

no_sub = []
for num in range(len(test_sub['id'])):
    if test_sub['id'][num] not in df_test['id'].values:
        no_sub.append(test_sub['id'][num])

print(no_sub)

[23102, 24760]


In [263]:
# 위의 'id' 값의 index 확인

for num in range(len(test_sub.values)):
    if test_sub['id'].values[num] in no_sub:
        print('index : ', test_sub.index[num], 'value : ', test_sub['id'].values[num])

index :  15947 value :  23102
index :  17085 value :  24760


In [264]:
# 존재하지 않는 'id' index 제거

test_sub = test_sub.drop(17085, axis=0)
test_sub = test_sub.drop(15947, axis=0)

In [265]:
print(test_sub['id'].values[15945])
print(test_sub['id'].values[15946])
print(test_sub['id'].values[15947])
print(test_sub['id'].values[15948])

23098
23101
23103
23104


In [266]:
print(test_sub['id'].values[17083])
print(test_sub['id'].values[17084])
print(test_sub['id'].values[17085])
print(test_sub['id'].values[17086])

24759
24761
24762
24764


### Train data 'product_description' 내 html 제거

In [267]:
for num in range(len(df_train['product_description'])):
    if "<li>" in df_train['product_description'][num]:
        df_train['product_description'][num] = []

C:\Users\tjdal\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### corpus 제작

In [268]:
df_corpus = pd.concat([df_train, df_test], axis=0)
df_corpus = df_corpus.drop('median_relevance', axis = 1)

In [269]:
len(sorted(df_corpus['id']))

32668

### Question / Result 구분

In [270]:
def feature_merge(df_text):
    textdata = list(df_text.apply(lambda x:'%s %s' % (x['product_title'], x['product_description']), axis=1))
    return textdata

In [271]:
train_q = list(df_train['query'])
test_q = list(df_test['query'])

In [272]:
train_r =feature_merge(df_train)
test_r = feature_merge(df_test)

In [273]:
corpusdata = joblib.load('corpusdata.pkl')

In [274]:
corpusdata

['bridal shower decorations Accent Pillow with Heart Design - Red/Black Red satin accent pillow embroidered with a heart in black thread. 8" x 8".',
 'led christmas lights Set of 10 Battery Operated Multi LED Train Christmas Lights - Clear Wire Set of 10 Battery Operated Train Christmas Lights Item #X124210 Features: Color: multi-color bulbs with matching train light covers / clear wire Multi-color consists of red, green, blue and yellow bulbs Number of bulbs on string: 10 Bulb size: micro LED Spacing between bulbs: 6 inches Lighted length: 4.5 feet Total length: 5.5 feet 12 inch lead cord Additional product features: LED lights use 90% less energy Cool to the touch If one bulb burns out, the rest will stay lit Lights are equipped with Lamp Lock feature, which makes them replaceable, interchangeable and keeps them from falling out Requires 3 "AA" batteries (not included) Convenient on/off/timer switch located on battery pack Timer function on battery pack allows for 6 hours on and 18 h

### 토큰 생성

In [275]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [276]:
def word_token(data):
    for num in range(len(data)):
        data[num] = word_tokenize(data[num])

In [277]:
%%time
word_token(train_q)
word_token(train_r)
word_token(test_q)
word_token(test_r)
word_token(corpusdata)

Wall time: 1min 12s


### 소문자 변환

In [278]:
def data_join(list_data):
    for num in range(len(list_data)):
        list_data[num] = (" ").join(list_data[num])

In [279]:
%%time
data_join(train_q)
data_join(train_r)
data_join(test_q)
data_join(test_r)
data_join(corpusdata)

Wall time: 485 ms


In [280]:
def lower_convert(data):
    for num in range(len(data)):
        data[num] = data[num].lower()

In [281]:
%%time
lower_convert(train_q)
lower_convert(train_r)
lower_convert(test_q)
lower_convert(test_r)
lower_convert(corpusdata)

Wall time: 154 ms


### Stopwords (2글자 이상 알파벳)

In [282]:
import re

In [283]:
def alphabet_stopwords(data):
    for num in range(len(data)):
        data[num] = re.findall(r'[a-zA-Z]+', data[num])
        
    for num in range(len(data)):
        data[num] = (" ").join(data[num])
    
    for num in range(len(data)):
        data[num] = re.findall(r'\w\w+', data[num])

In [284]:
%%time
alphabet_stopwords(train_q)
alphabet_stopwords(train_r)
alphabet_stopwords(test_q)
alphabet_stopwords(test_r)
alphabet_stopwords(corpusdata)

Wall time: 5.99 s


### Stopwords(nltk english)

In [285]:
from nltk.corpus import stopwords

In [286]:
def eng_stopwords(data):
    data_copy = data.copy()
    for num in range(len(data_copy)):
        data_copy[num] = []
    
    stop = stopwords.words('english')
    
    for num in range(len(data)):
        for i in range(len(data[num])):
            if data[num][i] not in stop:
                data_copy[num].append(data[num][i])
                
    return data_copy

In [287]:
%%time
train_q = eng_stopwords(train_q)
train_r = eng_stopwords(train_r)
test_q = eng_stopwords(test_q)
test_r = eng_stopwords(test_r)
corpusdata = eng_stopwords(corpusdata)

Wall time: 16.5 s


### Stemming

In [288]:
from nltk.stem import PorterStemmer

In [289]:
# 타 참가자의 코드 참고

def stemPorter(text):
            porter = PorterStemmer()
            stem_data = []
            for num in text:
                final_stem = []
                for word in num:
                    final_stem.append(porter.stem(word))
                stem_data.append(final_stem)
            return stem_data

In [290]:
%%time
train_q = stemPorter(train_q)
train_r = stemPorter(train_r)
test_q = stemPorter(test_q)
test_t = stemPorter(test_r)
corpus_data = stemPorter(corpusdata)

Wall time: 1min 57s


### TF-IDF / SVD

In [291]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [292]:
%%time
data_join(train_q)
data_join(train_r)
data_join(test_q)
data_join(test_r)
data_join(corpus_data)

Wall time: 307 ms


In [293]:
tfv = TfidfVectorizer(min_df=3, analyzer='word', ngram_range=(1, 3))

In [294]:
tfv.fit(corpus_data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [295]:
train_Q = tfv.transform(train_q)
train_R = tfv.transform(train_r)
test_Q = tfv.transform(test_q)
test_R = tfv.transform(test_r)

In [296]:
from scipy.sparse import hstack

In [297]:
train_X = hstack((train_Q, train_R))

In [298]:
test_X = hstack((test_Q, test_R))

In [299]:
len(train_q)

10157

In [300]:
len(y)

10157

In [301]:
print(train_Q.shape)
print(train_R.shape)
print(test_Q.shape)
print(test_R.shape)
print(train_X.shape)
print(test_X.shape)

(10157, 228261)
(10157, 228261)
(22511, 228261)
(22511, 228261)
(10157, 456522)
(22511, 456522)


### Modeling

In [302]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import pipeline
from sklearn import grid_search
from sklearn import metrics

In [250]:
svd = TruncatedSVD(n_components=400)

In [303]:
svd = TruncatedSVD()
scl = StandardScaler()
xgb_model = xgb.XGBRegressor()    

In [304]:
clf = pipeline.Pipeline([('svd', svd), ('scl', scl), ('xgb', xgb_model)])

In [159]:
param_grid = {'svd__n_components':[400]}

In [305]:
param_grid = {'xgb__max_depth':[5, 6, 7, 8, 9, 10], 'xgb__n_estimators':list(range(10, 101, 10)), 
              'xgb__learning_rate':[0.001, 0.01, 0.1], 'xgb__gamma':[0.1, 1.0, 10.0, 100.0], 'xgb__subsample':[0.75]}

In [309]:
model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

In [307]:
from ml_metrics import quadratic_weighted_kappa

In [308]:
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

In [105]:
y = joblib.load('y.pkl')

In [106]:
len(y)

10157

In [ ]:
train_X.shape

In [ ]:
%%time
model.fit(train_X, y)

Fitting 2 folds for each of 720 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   14.7s


In [90]:
model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=4)

In [91]:
%%time
reg_model = model.fit(train_X, y)

Fitting 4 folds for each of 720 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.1min
[Paralle

Wall time: 56min 59s


In [92]:
reg_model.best_score_

0.08146681133769301

In [ ]:
from sklearn.metrics import r2_score

In [252]:
model2 = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, verbose=1, n_jobs=-1, cv=10)

In [253]:
%%time
reg_model2 = model2.fit(train_X, y)

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\tjdal\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\tjdal\Anaconda3\lib\runpy.

In [116]:
model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, verbose=1, n_jobs=-1, cv=10)

In [117]:
%%time
reg_model = model.fit(train_X, y)

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 30.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 41.3min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 163.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 179.8min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 196.3min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 214.1min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 234.4min
[Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed: 234.6min finished


Wall time: 3h 54min 38s


In [121]:
reg_model.best_score_

0.04311341038698352

In [122]:
reg_model.best_params_

{'xgb__gamma': 1.0,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 5,
 'xgb__n_estimators': 60,
 'xgb__subsample': 0.75}